In [ ]:
import os

# Caminho raiz
raiz = os.getcwd()

# Lista de nomes de diretórios que você quer ignorar
excluir = {"venv"}

for dirpath, dirnames, filenames in os.walk(raiz, topdown=True):
    # Filtra dirnames para não conter os excluídos
    dirnames[:] = [d for d in dirnames if d not in excluir]

    # Agora apenas os diretórios permitidos serão visitados
    print(f"Visitando: {dirpath}")
    print(f"  Subdiretórios válidos: {dirnames}")
    print(f"  Arquivos: {filenames}")
    print("---")

Visitando: c:\Users\Logic\Desktop\Meu\Flask_integrações
  Subdiretórios válidos: ['app']
  Arquivos: ['.env', 'comandos.txt', 'rascunhos.ipynb', 'requirements.txt', 'run.py']
---
Visitando: c:\Users\Logic\Desktop\Meu\Flask_integrações\app
  Subdiretórios válidos: ['models', 'routes', 'services']
  Arquivos: ['config.py', 'utils.py', '__init.py']
---
Visitando: c:\Users\Logic\Desktop\Meu\Flask_integrações\app\models
  Subdiretórios válidos: []
  Arquivos: ['__init.py']
---
Visitando: c:\Users\Logic\Desktop\Meu\Flask_integrações\app\routes
  Subdiretórios válidos: []
  Arquivos: ['api_routes.py', 'webhook_routes.py', '__init__.py']
---
Visitando: c:\Users\Logic\Desktop\Meu\Flask_integrações\app\services
  Subdiretórios válidos: []
  Arquivos: ['tunnel_service.py', 'webhook_services.py', '__init.py']
---


In [3]:
import re

cnpj_str = "42.934.541/0001-37"

cnpj_int = re.sub(pattern=r"[\.\/-]", repl="", string=cnpj_str)

print(cnpj_int)

42934541000137


# Consultas dos campos dentro do Bitrix

In [1]:
import requests
import json


def get_company_fields():
    url = "https://logic.bitrix24.com.br/rest/260/af4o31dew3vzuphs/crm.company.fields"

    try:
        response = requests.get(url)
        response.raise_for_status()

        return response.json()

    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None


print(json.dumps(get_company_fields(), indent=2, ensure_ascii=False))

{
  "result": {
    "ID": {
      "type": "integer",
      "isRequired": false,
      "isReadOnly": true,
      "isImmutable": false,
      "isMultiple": false,
      "isDynamic": false,
      "title": "ID"
    },
    "TITLE": {
      "type": "string",
      "isRequired": true,
      "isReadOnly": false,
      "isImmutable": false,
      "isMultiple": false,
      "isDynamic": false,
      "title": "Nome da Empresa"
    },
    "COMPANY_TYPE": {
      "type": "crm_status",
      "isRequired": false,
      "isReadOnly": false,
      "isImmutable": false,
      "isMultiple": false,
      "isDynamic": false,
      "statusType": "COMPANY_TYPE",
      "title": "Tipo de empresa"
    },
    "LOGO": {
      "type": "file",
      "isRequired": false,
      "isReadOnly": false,
      "isImmutable": false,
      "isMultiple": false,
      "isDynamic": false,
      "title": "Logotipo"
    },
    "ADDRESS": {
      "type": "string",
      "isRequired": false,
      "isReadOnly": false,
      "isImmu

# Services para consultar o CNPJ na receta WS via API e dar um UPDATE nos dados da empresa

In [4]:
import os
import re
import json
import requests
import pandas as pd

In [ ]:
def get_cnpj_receita(cnpj: str) -> None:
    cnpj_int = re.sub(pattern=r"[\.\/-]", repl="", string=str(cnpj))

    url = f"https://publica.cnpj.ws/cnpj/{cnpj_int}"

    try:
        response = requests.get(url=url)
        response.raise_for_status()

        data = response.json()

        if "error" in data:
            print(f"❌ Erro na requisição API:\n{data['error']}\n")
        else:
            print(f"\n✅ Sucesso na consulta do CNPJ {cnpj}\n")
            with open("CNPJ_teste.json", "w") as file:
                file.write(json.dumps(data, indent=2, ensure_ascii=False))
                file.close()

            return data

    except requests.exceptions.RequestException as e:
        print(f"❌ Exceção na requisição API:\n{e}\n")


cnpj = "42.934.541/0001-37"
get_cnpj_receita(cnpj)


✅ Sucesso na consulta do CNPJ 42.934.541/0001-37



{'cnpj_raiz': '42934541',
 'razao_social': 'RAVC PARTICIPACOES LTDA',
 'capital_social': '100000.00',
 'responsavel_federativo': '',
 'atualizado_em': '2025-05-10T03:00:00.000Z',
 'porte': {'id': '01', 'descricao': 'Micro Empresa'},
 'natureza_juridica': {'id': '2062',
  'descricao': 'Sociedade Empresária Limitada'},
 'qualificacao_do_responsavel': {'id': 49,
  'descricao': 'Sócio-Administrador '},
 'socios': [{'cpf_cnpj_socio': '***097411**',
   'nome': 'ANNA CAROLINA FONSECA BAILONI FERNANDES',
   'tipo': 'Pessoa Física',
   'data_entrada': '2021-07-30',
   'cpf_representante_legal': '***000000**',
   'nome_representante': None,
   'faixa_etaria': '31 a 40 anos',
   'atualizado_em': '2025-02-08T03:00:00.000Z',
   'pais_id': '1058',
   'qualificacao_socio': {'id': 49, 'descricao': 'Sócio-Administrador '},
   'qualificacao_representante': None,
   'pais': {'id': '1058',
    'iso2': 'BR',
    'iso3': 'BRA',
    'nome': 'Brasil',
    'comex_id': '105'}},
  {'cpf_cnpj_socio': '***771921**

In [ ]:
def update_company_process_cnpj(raw_cnpj_json: dict) -> dict:
    # Mapeamento dos campos do CRM
    fields_mapping = {
        "UF_CRM_1708977581412": "cnpj",
        "TITLE": "nome",
        "UF_CRM_1709838249844": "nome_fantasia",
        "UF_CRM_ADDRESS": "endereco",
        "UF_CRM_1747351639": "cep",
        "UF_CRM_1710938520402": "inscricao_estadual",
    }

    # Extrair dados do estabelecimento
    company = raw_cnpj_json.get("estabelecimento", {})

    # Construir endereço completo
    endereco_completo = (
        f"{company.get('tipo_logradouro', '')} {company.get('logradouro', '')}, "
        f"{company.get('numero', '')} - {company.get('bairro', '')}"
    )

    # Mapear valores para os campos do CRM
    processed_data = {
        fields_mapping["UF_CRM_1708977581412"]: re.sub(
            r"(\d{2})(\d{3})(\d{3})(\d{4})(\d{2})",
            r"\1.\2.\3/\4-\5",
            company.get("cnpj", ""),
        ),
        fields_mapping["TITLE"]: raw_cnpj_json.get("razao_social", ""),
        fields_mapping["UF_CRM_1709838249844"]: company.get("nome_fantasia", ""),
        fields_mapping["UF_CRM_ADDRESS"]: endereco_completo.strip(),
        fields_mapping["UF_CRM_1747351639"]: company.get("cep", ""),
        fields_mapping["UF_CRM_1710938520402"]: (
            company.get("inscricoes_estaduais", [""])[0]
            if company.get("inscricoes_estaduais", [""])[0] is not None
            else "Não Contribuinte"
        ),
    }

    return {"crm_fields": processed_data}


def post_destination_api(processed_data: dict, api_url: str) -> dict:
    """
    Envia os dados processados para a API de destino

    Parameters:
        processed_data (dict): Dados formatados pelo process_cnpj_data
        api_url (str): URL da API de destino

    Returns:
        dict: Resposta da API
    """
    try:
        response = requests.post(api_url, json=processed_data["crm_fields"], timeout=10)

        response.raise_for_status()
        return response.json()

    except requests.exceptions.RequestException as e:
        print(f"❌ Erro na requisição: {str(e)}")
        return {"error": str(e)}